# FInDe

## Availables banks
- Cajamar
- Openbank
- Unicaja

## Fist step
Put your csv or xls files inside ```accounts/current/<bank_name>/file``` and ```accounts/savings/<bank_name>/file```

## Requisites

In [ ]:
!pip install plotly

## Code

### Imports

In [ ]:
import plotly as py
import plotly.graph_objs as go

import numpy as np
import datetime as dt
import os.path
import xlrd
import pandas as pd

py.offline.init_notebook_mode(connected=True)

### Process CSV and XLS

In [ ]:
def processCSV(file, splitToken, bank): #TODO Move to pandas
    fecha = []
    saldo = []
    first = True
    
    if bank == 'cajamar':
        with open(file) as f:
            for line in f:
                if first is False:
                    tokens = line.split(splitToken)
                    fecha.append(dt.datetime.strptime(tokens[0], '%d/%m/%Y').date())
                    saldo.append(float(tokens[len(tokens)-1][:-1].replace('.', '').replace(',', '.')))
                first = False

    return (saldo, fecha)

In [ ]:
def processXLS(file, bank):   
    balance = []
    date = []
    
    if bank == 'unicaja':
        book = xlrd.open_workbook(file)
        first_sheet = book.sheet_by_index(0)
        date_int = first_sheet.col_values(0)[5:]
        balance = first_sheet.col_values(5)[5:]
        date = [xlrd.xldate_as_datetime(date_int[i], book.datemode).date().isoformat() for i in  xrange(0, len(date_int))]
    
    if bank == 'openbank':
        df = pd.read_html(datafile)
        df = df[0].dropna(axis=0, thresh=4)
        dates = df.iloc[1:,1]
        balance = df.iloc[1:,9]

    return (balance, date)

### Traces

In [ ]:
def builtTraces(bankdata):
    data = []
    for bank in bankdata: # Bank name
        dates = np.array(bankdata[bank]['date'])
        balances = np.array(bankdata[bank]['balance'])
        
        trace_main = go.Scatter(
            x = dates,
            y = balances,
            name = str(bank) + ': Saldo ' + str(format(balances[-1], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
            line = dict(
                color = 'green'
            ),
            mode = 'lines'
        )
        data.push(trace_main)
        
        trace_max = go.Scatter(
            x = dates,
            y = [balances.max() for f in dates],
            name = str(bank) + ': Saldo máximo',
            #visible = 'legendonly',
            #hoverinfo = 'name',
            line = dict(
                color = 'cyan',
                width = 4,
                dash = 'dot'
            )
        )
        data.push(trace_max)
        
        trace_mean = go.Scatter(
            x = dates,
            y = [balances.mean() for f in dates],
            name = str(bank) + ': Saldo medio',
            #hoverinfo = 'none',
            line = dict(
                color = 'magenta',
                width = 4,
                dash = 'dashdot'
            )
        )
        data.push(trace_mean)
        
        trace_min = go.Scatter(
            x = dates,
            y = [balances.min() for f in dates],
            name = str(bank) + ': Saldo mínimo',
            line = dict(
                color = 'red',
                width = 4,
                dash = 'dot'
            )
        )
        data.push(trace_min)  
    return data

In [ ]:
print("Files found:")
path = "accounts/current/"
directories = [f for f in os.listdir(path) if os.path.isdir(path+f)]
bankdata = {}
for bankname in directories:
    bankdata[bankname] = {"date":[], "balance":[]}
    files = [os.path.join(path+bankname,f) for f in os.listdir(path+bankname)]
    files.sort(key=lambda x: os.path.getmtime(x))
    for datafile in files:
        extension = os.path.splitext(datafile)[1]
        if extension == ".xls":
            print("> " + bankname + "/" + datafile)            
            (balance, date) = processXLS(datafile, bankname)
            bankdata[bankname]["balance"].extend(balance)
            bankdata[bankname]["date"].extend(date)
        if extension == ".csv":
            print("> " + bankname + "/" + datafile)            
            (balance, date) = processCSV(datafile, '\t', bankname)
            bankdata[bankname]["balance"].extend(balance)
            bankdata[bankname]["date"].extend(date)

data = builtTraces(bankdata)

#### Cajamar y Openbank

In [ ]:
trace_main_cajamar = go.Scatter(
    x = fechasCajamar,
    y = saldosCajamar,
    name = 'Cajamar: Saldo ' + str(format(saldosCajamar[-1], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
    line = dict(
        color = 'green'
    ),
    mode = 'lines'
)
trace_max_cajamar = go.Scatter(
    x = fechasCajamar,
    y = [saldosCajamar.max() for f in fechasCajamar],
    name = 'Cajamar: Saldo máximo',
    #visible = 'legendonly',
    #hoverinfo = 'name',
    line = dict(
        color = 'cyan',
        width = 4,
        dash = 'dot'
    )
)
trace_mean_cajamar = go.Scatter(
    x = fechasCajamar,
    y = [saldosCajamar.mean() for f in fechasCajamar],
    name = 'Cajamar: Saldo medio',
    #hoverinfo = 'none',
    line = dict(
        color = 'magenta',
        width = 4,
        dash = 'dashdot'
    )
)
trace_min_cajamar = go.Scatter(
    x = fechasCajamar,
    y = [saldosCajamar.min() for f in fechasCajamar],
    name = 'Cajamar: Saldo mínimo',
    line = dict(
        color = 'red',
        width = 4,
        dash = 'dot'
    )
)

trace_main_openAhorro = go.Scatter(
    x = fechasOpenAhorro,
    y = saldosOpenAhorro,
    name = 'OpenAhorro: Saldo ' + str(format(saldosOpenAhorro[0], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
    line = dict(
        color = 'blue'
    ),
    mode = 'lines'
)

trace_main_openNomina = go.Scatter(
    x = fechasOpenNomina,
    y = saldosOpenNomina,
    name = 'OpenNomina: Saldo ' + str(format(saldosOpenNomina[0], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
    line = dict(
        color = 'darkred'
    ),
    mode = 'lines'
)

data = [trace_main_cajamar, trace_max_cajamar, trace_mean_cajamar, trace_min_cajamar,
        trace_main_openAhorro, trace_main_openNomina]

### Plot

#### Cajamar y Openbank

In [ ]:
layout = go.Layout(title = 'Saldo ' + str(fechasCaja[-1]) + ' / ' + str(fechasOpen[0]),
              xaxis = dict(title = 'Fecha'),
              yaxis = dict(title = 'Saldo (€)'),
              showlegend = True
)

fig = dict(data=data, layout=layout)
py.offline.iplot(fig, filename='styled-line')

### Full Unicaja Auto

In [ ]:
# CONSTANTS
CURRENT = "current"
SAVING = "saving"
ACCOUNTTYPES  = [CURRENT, SAVING]
UNICAJA = "unicaja"

In [ ]:
def sortDataFiles(bankdata):
    for accountType in ACCOUNTTYPES:
        for bankname in bankdata[accountType]:
            (bankdata[accountType][bankname]["date"],bankdata[accountType][bankname]["balance"]) =zip(*sorted(zip(bankdata[accountType][bankname]["date"], bankdata[accountType][bankname]["balance"])))
            
    return bankdata
            

In [ ]:
def loadDataFiles():    
    print("Data files found:")
    bankdata = {}
    for accountType in ACCOUNTTYPES:
        path = "accounts/{}/".format(accountType)
        directories = [f for f in os.listdir(path) if os.path.isdir(path+f)]
        bankdata[accountType] = {}
        for bankname in directories:
            bankdata[accountType][bankname] = {"date":[], "balance":[]}
            files = [os.path.join(path+bankname,f) for f in os.listdir(path+bankname)]
            #files.sort(key=lambda x: os.path.getmtime(x))
            for datafile in files:
                extension = os.path.splitext(datafile)[1]
                if extension == ".xls":
                    print("/t{}/{}".format(bankname, datafile))
                    (balance, date) = processXLS(datafile, bankname)
                    bankdata[accountType][bankname]["balance"].extend(balance)
                    bankdata[accountType][bankname]["date"].extend(date)
    return sortDataFiles(bankdata)

In [ ]:
bankdata = loadDataFiles()

fechas = bankdata[ACCOUNTTYPES[0]][UNICAJA]["date"]
saldos = bankdata[ACCOUNTTYPES[0]][UNICAJA]["balance"]

trace_main_unicaja = go.Scatter(
    x = fechas,
    y = saldos,
    name = 'Unicaja: Saldo ' + str(format(saldos[-1], ',.2f').replace(",", "X").replace(".", ",").replace("X", ".")) + '€',
    line = dict(
        color = 'green'
    ),
    mode = 'lines'
)

data = [trace_main_unicaja]

layout = go.Layout(title = 'Saldo ' + str(fechas[-1]),
              xaxis = dict(title = 'Fecha'),
              yaxis = dict(title = 'Saldo (€)'),
              showlegend = True
)
fig = dict(data=data, layout=layout)
py.offline.iplot(fig)